In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Eight_Metrics.csv')

df = df.dropna(subset=['token'])

df['nSFD'] = df.loc[:,'SFD'] / df['length']
df['nFFD'] = df.loc[:,'FFD'] / df['length']
df['nGD'] = df.loc[:,'GD'] / df['length']
df['nTT'] = df.loc[:,'TT'] / df['length']

df = df.astype({'subject': str})

df.head()

,file,line,part,token,subject,SFD,FFD,GD,TT,PrF,Pr1,Pr2,PrS,freq,length,nSFD,nFFD,nGD,nTT
0,11,12,3,brought,1,NaN,120.0,120.0,836.0,0.0,0.0,0.0,1.0,50,7,NaN,17.142857,17.142857,119.428571
1,11,13,3,broker,1,NaN,120.0,120.0,700.0,0.0,0.0,0.0,1.0,3,6,NaN,20.000000,20.000000,116.666667
2,11,12,4,to,1,NaN,161.0,161.0,353.0,0.0,0.0,0.0,1.0,19942,2,NaN,80.500000,80.500000,176.500000
3,11,13,4,to,1,161.0,161.0,161.0,161.0,0.0,0.0,0.0,1.0,19942,2,80.5,80.500000,80.500000,80.500000
4,11,12,5,arrange,1,NaN,74.0,74.0,695.0,0.0,0.0,0.0,1.0,7,7,NaN,10.571429,10.571429,99.285714


In [3]:
columns = 'nFFD nGD nTT PrF Pr1 Pr2 PrS freq length subject'.split() # nSFD was dropped
data = df[columns]

train, test = train_test_split(data, train_size=0.7)

train_target = train['subject']
train_data = train.drop(["subject"], axis=1)

test_target = test['subject']
test_data = test.drop(["subject"], axis=1)

In [4]:
scaler = MinMaxScaler()

train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns, index=train_data.index)

test_data = pd.DataFrame(scaler.fit_transform(test_data), columns=test_data.columns, index=test_data.index)

In [15]:
train_target

98274     146
142624    259
115257    287
118493    335
76564     306
         ... 
11313      43
210331    332
151326     41
17035      67
176857     43
Name: subject, Length: 161386, dtype: object

In [11]:
print("train data:", train_data.shape)
print("train data:", test_data.shape)

train data: (161386, 9)
train data: (69166, 9)


In [12]:
print('train target:', train_target.shape)
print('test target:', test_target.shape)

train target: (161386,)
test target: (69166,)


In [13]:
clf = RandomForestClassifier()

clf.fit(train_data, train_target)

RandomForestClassifier()

In [16]:
import pickle

In [17]:
pickle.dump(clf, open('clf.sav', 'wb'))

OSError: [Errno 28] No space left on device

In [10]:
from sklearn.model_selection import cross_val_score
import statistics

statistics.mean(cross_val_score(clf, train_data, train_target, cv=10, scoring="accuracy"))

0.014387864056612343

In [15]:
from sklearn.model_selection import cross_val_predict

test_predict = cross_val_predict(clf, test_data, test_target, cv=10)

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision_score(test_target, test_predict, average='micro')

0.012318191018708614

In [27]:
recall_score(test_target, test_predict, average='weighted')

0.012318191018708614

In [28]:
f1_score(test_target, test_predict, average='weighted')

0.012164997724418855

In [31]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_target, test_predict)

array([[62,  6,  1, ...,  1,  1,  4],
       [ 4,  7,  0, ...,  0,  2,  6],
       [ 1,  1,  1, ...,  0,  0,  1],
       ...,
       [ 0,  0,  0, ...,  0,  1,  0],
       [ 5,  1,  0, ...,  0,  6,  3],
       [ 3,  5,  1, ...,  0,  2,  0]])